## Business Objective

The goal of this analysis is to understand the drivers of Fulfillment Efficiency across
products, customers, and logistics operations, and to identify actionable levers that
can reduce returns, cancellations, and revenue loss.


## Defining Fulfillment Efficiency

Fulfillment Efficiency measures how much of the gross order value is ultimately realized
as delivered revenue after accounting for returns.


Fulfillment Efficiency = (Delivered Revenue − Returned Revenue) / Gross Order Value


## Key Metrics Used

- Gross Order Value (GOV)
- Delivered Revenue
- Returned Revenue
- Cancellation Rate
- Return Rate
- Fulfillment Efficiency


## Hypotheses

H1: Categories with lower product ratings have higher return rates.

H2: Products with low stock availability experience higher cancellation rates.

H3: Price deviations of ±10% from category averages are associated with poorer
fulfillment outcomes.

H4: Certain logistics combinations (shipping + payment method) drive higher returns.


In [1]:
import gdown
import pandas as pd

## Data Import

The following cells download raw datasets from cloud storage.
These are setup steps and not part of the analysis.


In [2]:
# @title
file_id = "1iFlv5PjnezdaCcTzWsjAX-Ck9kMCBMMK"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "orders.csv", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1iFlv5PjnezdaCcTzWsjAX-Ck9kMCBMMK
From (redirected): https://drive.google.com/uc?id=1iFlv5PjnezdaCcTzWsjAX-Ck9kMCBMMK&confirm=t&uuid=d41aafa4-252c-40f4-a027-fe1440c126de
To: /content/orders.csv
100%|██████████| 377M/377M [00:07<00:00, 48.9MB/s]


'orders.csv'

In [3]:
file_id = "1o25JTcxDBEaigjCrdq_bzKb9BtzCZdy8"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "orderline.csv", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1o25JTcxDBEaigjCrdq_bzKb9BtzCZdy8
From (redirected): https://drive.google.com/uc?id=1o25JTcxDBEaigjCrdq_bzKb9BtzCZdy8&confirm=t&uuid=74688600-0fb7-4926-a13c-6a7e9962ada6
To: /content/orderline.csv
100%|██████████| 642M/642M [00:12<00:00, 52.6MB/s]


'orderline.csv'

In [4]:
file_id = "1Aa5oSSE-3Fn6RQpupqcg2sAf3l2VlccA"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "person.csv", quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1Aa5oSSE-3Fn6RQpupqcg2sAf3l2VlccA
To: /content/person.csv
100%|██████████| 83.3M/83.3M [00:00<00:00, 121MB/s]


'person.csv'

In [5]:
file_id = "1dL388NuXzV8mpTJ44HEmp2LgxKR1d4z8"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, "product.csv", quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1dL388NuXzV8mpTJ44HEmp2LgxKR1d4z8
To: /content/product.csv
100%|██████████| 1.75M/1.75M [00:00<00:00, 61.3MB/s]


'product.csv'

In [6]:
df_person = pd.read_csv("person.csv", sep=";")
df_orders = pd.read_csv("orders.csv", sep=";")
df_orderline = pd.read_csv("orderline.csv", sep=";")
df_product = pd.read_csv("product.csv", sep=";")

In [7]:
df_orders.shape
df_orderline.shape
df_product.shape
df_person.shape

(600000, 12)

At this stage, no data cleaning was required as IDs and categorical fields were already well structured.

## Initial Data Inspection

To get a quick sense of the structure and fields in each table, I reviewed a small sample of rows from all datasets.  
This helps confirm column meanings and spot any obvious problems in data early.

In [ ]:
df_person['person_id'].is_unique
df_orders['order_id'].is_unique
df_product['product_id'].is_unique

True

In [ ]:
df_orders['status'].value_counts()
df_orderline['status'].value_counts()

,count
status,
fulfilled,2602356
cancelled,2600481
returned,2599406
pending,2599167
shipped,2598590


In [10]:
success_lines = ['fulfilled']
loss_lines = ['returned']

delivered_revenue = df_orderline.loc[
    df_orderline['status'].isin(success_lines), 'subtotal'
].sum()

returned_revenue = df_orderline.loc[
    df_orderline['status'].isin(loss_lines), 'subtotal'
].sum()

GOV = df_orderline['subtotal'].sum()

NRR = delivered_revenue - returned_revenue
fulfillment_efficiency = NRR / GOV
NRR, fulfillment_efficiency


(np.float64(10433400.990003586), np.float64(0.00014825130582453696))

This represents the overall fulfillment efficiency across the platform.


In [15]:
df_combined = (
    df_orderline
        .merge(
            df_product[
                [
                    'product_id',
                    'category',
                    'rating_average',
                    'review_count',
                    'price',
                    'stock_quantity'
                ]
            ],
            on='product_id',
            how='left'
        )
)

In [13]:
category_summary = df_combined.groupby('category').agg(
    GOV=('subtotal','sum'),
    Delivered=('subtotal', lambda x: x[df_combined.loc[x.index,'status']=='fulfilled'].sum()),
    Returned=('subtotal', lambda x: x[df_combined.loc[x.index,'status']=='returned'].sum())
)

category_summary['Fulfillment_Efficiency'] = (
    (category_summary['Delivered'] - category_summary['Returned']) /
    category_summary['GOV']
)

category_summary.sort_values('Fulfillment_Efficiency')


,GOV,Delivered,Returned,Fulfillment_Efficiency
category,,,,
Beverages,1.472792e+09,2.925245e+08,2.977884e+08,-0.003574
Gaming,1.364663e+09,2.708127e+08,2.747989e+08,-0.002921
Kitchenware,1.174737e+09,2.326127e+08,2.356252e+08,-0.002564
Automotive,1.237309e+09,2.449236e+08,2.480962e+08,-0.002564
Reference,1.164652e+09,2.314727e+08,2.341926e+08,-0.002335
Car Electronics,1.259669e+09,2.518688e+08,2.545248e+08,-0.002108
Health,1.371456e+09,2.738700e+08,2.761382e+08,-0.001654
Audiobooks,1.510470e+09,3.007802e+08,3.029629e+08,-0.001445
Computing,1.534296e+09,3.069052e+08,3.088030e+08,-0.001237


#Fulfillment efficiency table
Several categories above show negative fulfillment efficiency due to return volumes exceeding fulfilled revenue. This indicates operational or product-level issues such as high return rates, customer dissatisfaction, or fragile logistics in those categories.

In [27]:
df_combined.groupby(pd.cut(df_combined['rating_average'], bins=5))['status'] \
           .value_counts(normalize=True)


/tmp/ipython-input-408004261.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_combined.groupby(pd.cut(df_combined['rating_average'], bins=5))['status'] \


rating_average  status   
(0.996, 1.8]    shipped      0.200196
                fulfilled    0.200077
                cancelled    0.199992
                pending      0.199898
                returned     0.199838
(1.8, 2.6]      returned     0.200352
                fulfilled    0.200138
                cancelled    0.199917
                pending      0.199875
                shipped      0.199718
(2.6, 3.4]      fulfilled    0.200456
                cancelled    0.200208
                pending      0.199804
                shipped      0.199804
                returned     0.199729
(3.4, 4.2]      cancelled    0.200279
                returned     0.200074
                shipped      0.199902
                pending      0.199899
                fulfilled    0.199845
(4.2, 5.0]      fulfilled    0.200419
                pending      0.200222
                shipped      0.199800
                cancelled    0.199782
                returned     0.199777
Name: proportion, dtype: float64

### Impact of Product Ratings on Fulfillment Outcomes

Across all rating bands, order status distributions remain nearly uniform.
This suggests that product ratings alone are not a strong predictor of
returns, cancellations, or fulfillment success in this dataset.

As a result, further analysis focuses on operational and pricing factors
that may have a stronger influence on fulfillment efficiency.

No meaningful relationship is observed between product rating and fulfillment status. Status distributions remain nearly uniform across all rating bands.

In [25]:
df_combined.groupby(pd.cut(df_combined['stock_quantity'], bins=5))['status'] \
           .value_counts(normalize=True)

/tmp/ipython-input-3651899574.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_combined.groupby(pd.cut(df_combined['stock_quantity'], bins=5))['status'] \


stock_quantity  status   
(-0.5, 100.0]   cancelled    0.200117
                returned     0.200098
                fulfilled    0.200061
                pending      0.199921
                shipped      0.199803
(100.0, 200.0]  shipped      0.200247
                returned     0.199950
                fulfilled    0.199939
                pending      0.199937
                cancelled    0.199928
(200.0, 300.0]  fulfilled    0.200754
                returned     0.200002
                shipped      0.199840
                cancelled    0.199729
                pending      0.199676
(300.0, 400.0]  cancelled    0.200277
                fulfilled    0.200164
                returned     0.199891
                shipped      0.199850
                pending      0.199819
(400.0, 500.0]  pending      0.200342
                cancelled    0.200153
                fulfilled    0.199974
                returned     0.199823
                shipped      0.199707
Name: proportion, dtype: float64